In [1]:
import ee
import pandas as pd
import geopandas as gpd
import json
import geemap

import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()


from parameters.config_output_naming import geo_id_column
from modules.agstack_to_gee import start_agstack_session, get_agstack_token, geometry_to_geo_id, geo_id_or_ids_to_feature_collection, register_feature_and_set_geo_id, register_fc_and_set_geo_id, shapefile_to_ee_feature_collection 
from parameters.config_asr_url_info import asset_registry_base,user_registry_base
from parameters.config_asr_credentials import email,password

def whisp_alt_stats_as_df (roi): 
    %store roi
    %run alt_whisp_stats.ipynb   
    return df_out

#fetch and convert into feature collection
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)


Earth Engine has been initialized with the specified credentials.


#### List of Geo_ids

In [2]:
GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

all_geo_ids= GHA_ids + IDN_ids + CIV_ids

#make into a feature collection
roi = geo_id_or_ids_to_feature_collection(
    all_geo_ids=all_geo_ids, 
    geo_id_column=geo_id_column, 
    session=session)

#Whisp stats as a dataframe
whisp_alt_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
stats time: 2.0716395378112793 seconds
ee to df processing time: 7.863019227981567 seconds
table tidying time: 0.024723529815673828 seconds


,Geo id,Area ha,Country,EUFO 2020,GLAD Primary,TMF undist,JAXA FNF 2020,GFC TC 2020,GLAD LULC 2020,ESA TC 2020,TMF disturbed,RADD alerts,TMF plant,Oil palm Descals,Oil palm FDaP,Cocoa ETH,WDPA
PLOTID,,,,,,,,,,,,,,,,,
1,88bec54ad04804f5b1fafbc131266640a129be2840fa67...,1.9,GHA,19,0,0,100,20,85,100,18,-,0,0,0,0,-
2,97408ef7bdac487e4a42e4abf20492b786310889fd4b04...,16.6,GHA,100,87,95,100,100,89,100,5,True,0,0,0,0,True
3,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...,31.2,IDN,0,0,0,6,1,71,93,0,-,100,99,82,0,-
4,8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53...,20.9,IDN,28,0,31,96,54,96,100,44,-,0,0,51,0,-
5,0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f66...,8.3,CIV,5,0,0,95,61,21,77,4,-,0,0,0,0,True
6,fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b...,3.6,CIV,41,0,0,100,46,100,98,6,-,0,0,0,0,-


#### Show map for drawing geometry

In [3]:
Map = geemap.Map(center=[6,-2], zoom=12)
sc = Sidecar(title='draw polygon')
with sc:display(Map)

#### User input needed: draw boundary on map 

#### Fetch geo id and Whisp stats

In [8]:
#teakse
drawn_feature = (Map.draw_last_feature)

#register geo id and add to feature
feature_w_geo_id = register_feature_and_set_geo_id(
    drawn_feature,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=True)

roi = feature_w_geo_id

whisp_alt_stats_as_df(ee.FeatureCollection(roi))


NameError: name 'Map' is not defined

#### Get Geo_id

In [2]:
geometry = ee.Geometry.Polygon([
    [-122.091, 37.418],
    [-122.086, 37.422],
    [-122.079, 37.425],
    [-122.086, 37.418]
])

# Convert the polygon to WKT using the function
geo_id = geometry_to_geo_id(geometry,token,session,asset_registry_base,debug=True)

# Print the WKT representation
print('geo_id representation:', geo_id)

using existing session
Failed to register field boundary as matched existing one. Status code: 400
Returning existing (registered) geo id for field boundary ['7a22c499d0f65751c3f0347763de13b117844c6a9fe09298abfca9e671e065e2']
geo_id representation: 7a22c499d0f65751c3f0347763de13b117844c6a9fe09298abfca9e671e065e2


In [12]:
# Define an example ee.Feature with a polygon geometry
feature = ee.Feature(ee.Geometry.Polygon([
    [-122.091, 37.418],
    [-122.086, 37.422],
    [-122.079, 37.425],
    [-122.086, 37.418]
]))
# add geo id (from asset registry) to feature. Registers field if not already in registry)
feature_w_properties = register_feature_and_set_geo_id(feature,geo_id_column,token,session,asset_registry_base,debug=True)

roi =ee.FeatureCollection(feature_w_properties)

whisp_alt_stats_as_df(roi)

using existing session
Failed to register field boundary as matched existing one. Status code: 400
Returning existing (registered) geo id for field boundary ['7a22c499d0f65751c3f0347763de13b117844c6a9fe09298abfca9e671e065e2']
Stored 'roi' (FeatureCollection)
stats time: 2.011880397796631 seconds
ee to df processing time: 6.369735956192017 seconds
table tidying time: 0.035372257232666016 seconds


,Geo id,Area ha,Country,EUFO 2020,GLAD Primary,TMF undist,JAXA FNF 2020,GFC TC 2020,GLAD LULC 2020,ESA TC 2020,TMF disturbed,RADD alerts,TMF plant,Oil palm Descals,Oil palm FDaP,Cocoa ETH,WDPA
PLOTID,,,,,,,,,,,,,,,,,
1,7a22c499d0f65751c3f0347763de13b117844c6a9fe092...,23.6,USA,0,0,0,0,13,0,22,0,-,0,0,0,0,-


#### From asset (feature collection) 

In [4]:
example_plot_feature_col_asset = "projects/fdap-remi/assets/selected_segments_peru_vector"
fc = ee.FeatureCollection(example_plot_feature_col_asset)

#run for feature collection
roi = register_fc_and_set_geo_id(
    fc,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=True
)

whisp_alt_stats_as_df(roi)

using existing session
Failed to register. Matched existing field. Status code: 400 Using pre-existing geo id:  1881ce94a698c1f4fd137c954321edfe12b55a29b091e97adc57013247df54ad
using existing session
Failed to register. Matched existing field. Status code: 400 Using pre-existing geo id:  612f277a1e74672ac680f217f4034ad7522912d0a153396fb4400ed1886ffe6c
using existing session
Failed to register. Matched existing field. Status code: 400 Using pre-existing geo id:  236fcc56dfde65a19e928bf908258bea9377ab287e6a090837dbbefe20a03470
using existing session
Failed to register. Matched existing field. Status code: 400 Using pre-existing geo id:  68f96530c30f62ebf61fc17fdc8bac45b8c221a86aff7e5cba446f3d5e9614c9
using existing session
Failed to register. Matched existing field. Status code: 400 Using pre-existing geo id:  2259777df372f11ae5b958e601406e1933eff88fe8bb11f3632dc28d01368c20
using existing session
Failed to register. Matched existing field. Status code: 400 Using pre-existing geo id:  046

,Geo id,Area ha,Country,EUFO 2020,GLAD Primary,TMF undist,JAXA FNF 2020,GFC TC 2020,GLAD LULC 2020,ESA TC 2020,TMF disturbed,RADD alerts,TMF plant,Oil palm Descals,Oil palm FDaP,Cocoa ETH,WDPA
PLOTID,,,,,,,,,,,,,,,,,
1,1881ce94a698c1f4fd137c954321edfe12b55a29b091e9...,3.8,PER,74,0,0,36,2,0,48,0,-,0,0,0,0,-
2,612f277a1e74672ac680f217f4034ad7522912d0a15339...,2.0,PER,47,0,0,59,10,0,58,0,-,0,0,0,0,-
3,236fcc56dfde65a19e928bf908258bea9377ab287e6a09...,0.1,PER,0,0,0,80,0,0,0,0,-,0,0,0,0,-
4,68f96530c30f62ebf61fc17fdc8bac45b8c221a86aff7e...,0.2,PER,2,0,0,0,9,0,40,0,-,0,0,0,0,-
5,2259777df372f11ae5b958e601406e1933eff88fe8bb11...,2.1,PER,1,0,0,5,1,0,6,0,-,0,0,0,0,-
6,046d70c59a280cd27ce544372bd683df0fb05d87c98689...,1.5,PER,22,0,0,52,0,0,43,0,-,0,0,0,0,-
7,9010a602e7e420f84d2bf22973c2286f358f7b411887cb...,1.0,PER,17,0,0,37,0,0,16,0,-,0,0,0,0,-
8,f21827a0ae40ec43c74343071db7a4bef54dfc8de655ae...,2.5,PER,0,0,0,67,0,0,6,0,-,0,0,0,0,-
9,64b5ec6f83e2e72a703960782d49a76844499469554691...,2.5,PER,4,0,0,57,0,0,4,0,-,0,0,0,0,-


#### Feature collection

In [5]:
geom_1 =ee.Geometry.Polygon(
        [[[102.19867776108482, 3.0835809888660344],
          [102.1988172359536, 3.082670357801012],
          [102.20001886559227, 3.082078035157836],
          [102.2013385124276, 3.081745922272887],
          [102.2014355657636, 3.0825815609469625],
          [102.20011591892828, 3.0836528916202233]]]);

geom_2 = ee.Geometry.Polygon(
        [[[102.18777620396249, 3.079459767638651],
          [102.18831264576546, 3.0785384196951506],
          [102.1895786484205, 3.0792026473624907],
          [102.18899929127328, 3.08005971470826]]],None, False);

feat_1 = ee.Feature(geom_1)
feat_2 = ee.Feature(geom_2)

fc = ee.FeatureCollection([feat_1,feat_2])

#run for feature collection
roi = register_fc_and_set_geo_id(
    fc,
    geo_id_column,
    token,
    session,
    asset_registry_base,
    debug=True
)
roi

whisp_alt_stats_as_df(roi)


using existing session
Failed to register. Matched existing field. Status code: 400 Using pre-existing geo id:  4c8eef8c50e8b26eaffc5857117c71cd64e0f259b33cb985b685a7dfb6516cbd
using existing session
Failed to register. Matched existing field. Status code: 400 Using pre-existing geo id:  fdf0e980c1ccff20bb84f182a8a7b1ae614ace03b926b5adde2f297205d3c1d3
Stored 'roi' (FeatureCollection)
stats time: 2.1559526920318604 seconds
ee to df processing time: 3.286931037902832 seconds
table tidying time: 0.011808156967163086 seconds


,Geo id,Area ha,Country,EUFO 2020,GLAD Primary,TMF undist,JAXA FNF 2020,GFC TC 2020,GLAD LULC 2020,ESA TC 2020,TMF disturbed,RADD alerts,TMF plant,Oil palm Descals,Oil palm FDaP,Cocoa ETH,WDPA
PLOTID,,,,,,,,,,,,,,,,,
1,4c8eef8c50e8b26eaffc5857117c71cd64e0f259b33cb9...,4.0,MYS,0,0,2,0,45,70,96,43,-,0,49,78,0,-
2,fdf0e980c1ccff20bb84f182a8a7b1ae614ace03b926b5...,1.8,MYS,4,27,15,16,31,97,97,74,True,0,0,36,0,-


#### Shapefile polygon

In [9]:
shp_path = "test_ceo_all.shp.zip"

# # Load your geospatial dataset
# gdf = gpd.read_file(shp)

# geo_json = gdf.to_json()

# roi = ee.FeatureCollection(json.loads(geo_json))

# roi = roi.select([geo_id_column])

# def shapefile_to_ee_feature_collection(shapefile_path):
#     """
#     Convert a zipped shapefile to an Earth Engine FeatureCollection.

#     Args:
#     - shapefile_path (str): Path to the shapefile (.zip) to be converted.
#     - geo_id_column (str): Name of the column to be used as the GeoID.

#     Returns:
#     - ee.FeatureCollection: Earth Engine FeatureCollection created from the shapefile.
#     """
#     # Unzip the shapefile
#     # with zipfile.ZipFile(shapefile_path, "r") as zip_ref:
#     #     zip_ref.extractall("shapefile")

#     # Load the shapefile into a GeoDataFrame
#     gdf = gpd.read_file(shapefile_path)#"shapefile/test_ceo_all.shp")

#     # Convert GeoDataFrame to GeoJSON
#     geo_json = gdf.to_json()

#     # Create a FeatureCollection from GeoJSON
#     roi = ee.FeatureCollection(json.loads(geo_json))

    # return roi
# geemap.shp_to_ee(shp)
roi = shapefile_to_ee_feature_collection(shp_path)

whisp_alt_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
stats time: 2.0782933235168457 seconds
ee to df processing time: 9.450502157211304 seconds
table tidying time: 0.018945932388305664 seconds


,Geo id,Area ha,Country,EUFO 2020,GLAD Primary,TMF undist,JAXA FNF 2020,GFC TC 2020,GLAD LULC 2020,ESA TC 2020,TMF disturbed,RADD alerts,TMF plant,Oil palm Descals,Oil palm FDaP,Cocoa ETH,WDPA
PLOTID,,,,,,,,,,,,,,,,,
1,88bec54ad04804f5b1fafbc131266640a129be2840fa67...,1.9,GHA,19,0,0,100,20,85,100,18,-,0,0,0,0,-
2,ef2f7c46fbe4fc892fdb81f9a31c9c507b9f1e45485042...,4.2,GHA,19,0,5,99,34,47,93,65,-,0,0,0,0,-
3,97408ef7bdac487e4a42e4abf20492b786310889fd4b04...,16.6,GHA,100,87,95,100,100,89,100,5,True,0,0,0,0,True
4,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...,31.2,IDN,0,0,0,6,1,71,93,0,-,100,99,82,0,-
5,1a41a309ae2387f36a604c9a6c81887e64357a7f61d228...,2.0,IDN,0,0,0,86,1,17,99,1,-,0,0,4,0,-
6,1a4472dc40700ef33f931863f58d444f243d6441861667...,12.7,IDN,63,0,60,100,83,98,100,36,-,0,0,69,0,-
7,8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53...,20.9,IDN,28,0,31,96,54,96,100,44,-,0,0,51,0,-
8,0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f66...,8.3,CIV,5,0,0,95,61,21,77,4,-,0,0,0,0,True
9,b84f55de2b7f3c77d1cbeb8b026a1b29be42d8b08d9205...,2.0,CIV,33,0,0,70,78,15,87,11,-,0,0,0,75,True


#### Points

In [ ]:
#get test points from polygon feature collection (agstack currently not working)

# Compute centroids of each polygon
def getCentroid(feature):
    keepProperties = [geo_id_column];
    # Get the centroid of the feature's geometry.
    centroid = feature.geometry().centroid(1);
    #Return a new Feature, copying properties from the old Feature.
    return ee.Feature(centroid).copyProperties(feature, keepProperties);

roi = ee.FeatureCollection(fc.map(getCentroid))


whisp_alt_stats_as_df(roi)

#### Export CSV

In [ ]:
#export output csv
out_file_wide = "alt_whisp_output_table.csv"

df_out.to_csv(path_or_buf=out_file_wide,header=True)

print ("output csv: ", out_file_wide)